In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
import numpy as np 
import matplotlib.pyplot as plt
import time
from math import floor

In [2]:
# Pre-setting dan Dataset MNIST
torch.random.manual_seed(1)

compose = transforms.Compose([
                transforms.ToTensor(),
                transforms.ColorJitter(brightness=0.5, contrast=0.2, saturation=0.2, hue=0.2), 
                transforms.Normalize(mean=(0.1307), std=(0.3079)),
            ])

mnist_train = MNIST(
                root=r'mnist_dataset/train', 
                train=True, 
                transform=compose,
                download=True
            )

mnist_test = MNIST(
                root=r'mnist_dataset/test', 
                train=False, 
                transform=compose,
                download=True
            )

trainloader = DataLoader(mnist_train, batch_size=100, shuffle=True)
testloader = DataLoader(mnist_test, batch_size=100, shuffle=True)

Alexnet + conv2d (biar ada 3 channel) + Linear layer + zero-padded input

In [3]:
from torchvision.models.alexnet import AlexNet
from tqdm import tqdm
import torchsummary as ts

# load model
preConv = nn.Conv2d(1, 3, kernel_size=3, stride=1, padding='same')
alexnet = AlexNet()
newFeatures = nn.Sequential(
    preConv,
    nn.ZeroPad2d(98),
    alexnet.features
)
newClassifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(9216, 1000, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(1000, 10, bias=True)
)

# new model
finetuned_satu = nn.Sequential(
    newFeatures,
    alexnet.avgpool,
    nn.Flatten(start_dim=1),
    newClassifier
    )
ts.summary(finetuned_satu, (1,28,28))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 3, 28, 28]              30
         ZeroPad2d-2          [-1, 3, 224, 224]               0
            Conv2d-3           [-1, 64, 55, 55]          23,296
              ReLU-4           [-1, 64, 55, 55]               0
         MaxPool2d-5           [-1, 64, 27, 27]               0
            Conv2d-6          [-1, 192, 27, 27]         307,392
              ReLU-7          [-1, 192, 27, 27]               0
         MaxPool2d-8          [-1, 192, 13, 13]               0
            Conv2d-9          [-1, 384, 13, 13]         663,936
             ReLU-10          [-1, 384, 13, 13]               0
           Conv2d-11          [-1, 256, 13, 13]         884,992
             ReLU-12          [-1, 256, 13, 13]               0
           Conv2d-13          [-1, 256, 13, 13]         590,080
             ReLU-14          [-1, 256,

In [4]:
# train loop

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(finetuned_satu.parameters(), lr=1e-4)

# loop over the dataset multiple times
device = torch.device('cpu')
finetuned_satu = finetuned_satu.to(device) 
max_train_batch = trainloader.__len__()

start = time.time()
epochs = 2
for epoch in range(epochs):
    # print(f"Epoch [{epoch+1}/{epochs}]")
    running_loss, running_accuracy = 0.0, 0.0
    loop = tqdm(trainloader)
    for i, data in enumerate(loop, 0):
        loop.set_description(f"Epoch [{epoch+1}/{epochs}]")
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = finetuned_satu(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            batch_acc = torch.sum(torch.argmax(outputs, dim=1) == labels) / labels.shape[0]
            running_accuracy += batch_acc.item()
            running_loss += loss.item()
        loop.set_postfix(loss = running_loss / (i+1), accuracy = (running_accuracy / (i+1)), refresh = True)
            
            # if ((i + 1) % (max_train_batch // 3) == 0) or (i + 1) == max_train_batch:
            #     print(f"  Batch [{i+1}/{max_train_batch}] Loss : {running_loss / (i+1):.4f} Accuracy : {(running_accuracy / (i + 1)):.4f}")

end = time.time()
print(f'Total training time  [{epochs} epochs : {(end - start):.4f} Second]')

Epoch [2/2]: 100%|██████████| 600/600 [16:46<00:00,  1.68s/it, accuracy=0.929, loss=0.234]

Total training time  [2 epochs : 1994.9525 Second]


In [5]:
# Save Model
torch.save(finetuned_satu, "Models/Alexnet1.pt")

Alexnet (pretrained)+ conv2d (biar ada 3 channel) + Linear layer + zero-padded input

In [6]:
from torchvision.models.alexnet import alexnet
from tqdm import tqdm

# load model
preConv = nn.Conv2d(1, 3, kernel_size=3, stride=1, padding='same')
net = alexnet(pretrained = True)
newFeatures = nn.Sequential(
    preConv,
    nn.ZeroPad2d(98),
    net.features
)
newClassifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(9216, 1000, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(1000, 10, bias=True)
)

# new model
finetuned_dua = nn.Sequential(
    newFeatures,
    net.avgpool,
    nn.Flatten(start_dim=1),
    newClassifier
    )
print(finetuned_dua)

/home/ble/miniconda3/envs/mnist/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ble/miniconda3/envs/mnist/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Sequential(
    (0): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ZeroPad2d((98, 98, 98, 98))
    (2): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (1): AdaptiveAvgPool2d(o

In [16]:
# train loop

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(finetuned_dua.parameters(), lr=1e-4)

# loop over the dataset multiple times
device = torch.device('cpu')
finetuned_dua = finetuned_dua.to(device) 
max_train_batch = trainloader.__len__()

start = time.time()
epochs = 2
for epoch in range(epochs):
    # print(f"Epoch [{epoch+1}/{epochs}]")
    running_loss, running_accuracy = 0.0, 0.0
    loop = tqdm(trainloader)
    for i, data in enumerate(loop, 0):
        loop.set_description(f"Epoch [{epoch+1}/{epochs}]")
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = finetuned_dua(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            batch_acc = torch.sum(torch.argmax(outputs, dim=1) == labels) / labels.shape[0]
            running_accuracy += batch_acc.item()
            running_loss += loss.item()
        loop.set_postfix(loss = running_loss / (i+1), accuracy = (running_accuracy / (i+1)), refresh = True)
            
end = time.time()
print(f'Total training time  [{epochs} epochs : {(end - start):.4f} Second]')

Epoch [2/2]:  85%|████████▌ | 511/600 [14:34<02:32,  1.71s/it, accuracy=0.975, loss=0.0802]


KeyboardInterrupt: 

In [18]:
# Save Model
torch.save(finetuned_dua, "Models/Alexnet2.pt")

Alexnet(5 feature layer) + conv2d (biar ada 3 channel) + Linear layer + zero-padded input

In [12]:
from torchvision.models.alexnet import alexnet

# load model
preConv = nn.Conv2d(1, 3, kernel_size=3, stride=1, padding='same')
net = alexnet(pretrained = True)
newFeatures = nn.Sequential(
    preConv,
    nn.ZeroPad2d(98),
    net.features[:6]
)
newClassifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(6912, 1000, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(1000, 10, bias=True)
)

# new model
finetuned_tiga = nn.Sequential(
    newFeatures,
    net.avgpool,
    nn.Flatten(start_dim=1),
    newClassifier
    )
print(finetuned_tiga)
fakeInput = torch.rand(1, 28,28)
print(finetuned_tiga[:2](fakeInput).shape)

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ZeroPad2d((98, 98, 98, 98))
    (2): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
  )
  (1): AdaptiveAvgPool2d(output_size=(6, 6))
  (2): Flatten(start_dim=1, end_dim=-1)
  (3): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=6912, out_features=1000, bias=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=1000, out_features=10, bias=True)
  )
)
torch.Size([192, 6, 6])


In [13]:
# train loop

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(finetuned_tiga.parameters(), lr=1e-4)

# loop over the dataset multiple times
device = torch.device('cpu')
finetuned_tiga = finetuned_tiga.to(device) 
max_train_batch = trainloader.__len__()

start = time.time()
epochs = 2
for epoch in range(epochs):
    # print(f"Epoch [{epoch+1}/{epochs}]")
    running_loss, running_accuracy = 0.0, 0.0
    loop = tqdm(trainloader)
    for i, data in enumerate(loop, 0):
        loop.set_description(f"Epoch [{epoch+1}/{epochs}]")
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = finetuned_tiga(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            batch_acc = torch.sum(torch.argmax(outputs, dim=1) == labels) / labels.shape[0]
            running_accuracy += batch_acc.item()
            running_loss += loss.item()
        loop.set_postfix(loss = running_loss / (i+1), accuracy = (running_accuracy / (i+1)), refresh = True)
            
            # if ((i + 1) % (max_train_batch // 3) == 0) or (i + 1) == max_train_batch:
            #     print(f"  Batch [{i+1}/{max_train_batch}] Loss : {running_loss / (i+1):.4f} Accuracy : {(running_accuracy / (i + 1)):.4f}")

end = time.time()
print(f'Total training time  [{epochs} epochs : {(end - start):.4f} Second]')

Epoch [2/2]: 100%|██████████| 600/600 [10:36<00:00,  1.06s/it, accuracy=0.956, loss=0.15] 

Total training time  [2 epochs : 1256.9224 Second]


In [14]:
# Save Model
torch.save(finetuned_tiga, "Models/Alexnet3.pt")

Alexnet(5 feature layer) + conv2d (biar ada 3 channel) + Linear layer

In [16]:
from torchvision.models.alexnet import alexnet

# load model
preConv = nn.Conv2d(1, 3, kernel_size=3, stride=1, padding='same')
net = alexnet(pretrained = True)
newFeatures = nn.Sequential(
    preConv,
    net.features[:5]
)
newClassifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(768, 100, bias=True),
    nn.ReLU(inplace=True),
    nn.Linear(100, 10, bias=True)
)
net.avgpool = nn.AdaptiveAvgPool2d(output_size=(2,2))


# new model
finetuned_empat = nn.Sequential(
    newFeatures,
    net.avgpool,
    nn.Flatten(start_dim=1),
    newClassifier
    )
print(finetuned_empat)
fakeInput = torch.rand(1, 28,28)
print(finetuned_empat[:2](fakeInput).shape)

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
    )
  )
  (1): AdaptiveAvgPool2d(output_size=(2, 2))
  (2): Flatten(start_dim=1, end_dim=-1)
  (3): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=768, out_features=100, bias=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=100, out_features=10, bias=True)
  )
)
torch.Size([192, 2, 2])


In [ ]:
# train loop

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(finetuned_empat.parameters(), lr=1e-4)

# loop over the dataset multiple times
device = torch.device('cpu')
finetuned_empat = finetuned_empat.to(device) 
max_train_batch = trainloader.__len__()

start = time.time()
epochs = 10
for epoch in range(epochs):
    # print(f"Epoch [{epoch+1}/{epochs}]")
    running_loss, running_accuracy = 0.0, 0.0
    loop = tqdm(trainloader)
    for i, data in enumerate(loop, 0):
        loop.set_description(f"Epoch [{epoch+1}/{epochs}]")
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        
        # forward + backward + optimize
        outputs = finetuned_empat(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        with torch.no_grad():
            batch_acc = torch.sum(torch.argmax(outputs, dim=1) == labels) / labels.shape[0]
            running_accuracy += batch_acc.item()
            running_loss += loss.item()
        loop.set_postfix(loss = running_loss / (i+1), accuracy = (running_accuracy / (i+1)), refresh = True)
            
            # if ((i + 1) % (max_train_batch // 3) == 0) or (i + 1) == max_train_batch:
            #     print(f"  Batch [{i+1}/{max_train_batch}] Loss : {running_loss / (i+1):.4f} Accuracy : {(running_accuracy / (i + 1)):.4f}")

end = time.time()
print(f'Total training time  [{epochs} epochs : {(end - start):.4f} Second]')

Epoch [10/10]: 100%|██████████| 600/600 [00:26<00:00, 22.54it/s, accuracy=0.975, loss=0.0841]

Total training time  [10 epochs : 267.2180 Second]


In [ ]:
# Save Model
torch.save(finetuned_empat.state_dict(), "Models/Alexnet4.pt")

Best Final Performance: Alexnet(5 feature layer) + conv2d (biar ada 3 channel) + Linear layer + No Padding